# Lab 15: Lateral Movement Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab15_lateral_movement.ipynb)

Detect adversary movement through enterprise networks.

## Learning Objectives
- Authentication anomaly detection
- Remote execution identification (PsExec, WMI, WinRM)
- Graph-based attack path analysis
- Windows security event correlation

In [ ]:
!pip install pandas networkx matplotlib anthropic -q

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from datetime import datetime, timedelta
from typing import List, Dict

## 1. Authentication Event Analysis

In [ ]:
# Sample Windows authentication events
AUTH_EVENTS = [
    {"event_id": 4624, "user": "admin", "src_host": "WS001", "dst_host": "DC01", "logon_type": 3, "time": "2024-01-15 10:00:00"},
    {"event_id": 4624, "user": "admin", "src_host": "WS001", "dst_host": "FS01", "logon_type": 3, "time": "2024-01-15 10:00:30"},
    {"event_id": 4624, "user": "admin", "src_host": "FS01", "dst_host": "DB01", "logon_type": 3, "time": "2024-01-15 10:01:00"},  # Suspicious: FS01 as source
    {"event_id": 4625, "user": "admin", "src_host": "WS001", "dst_host": "DB01", "logon_type": 3, "time": "2024-01-15 10:02:00"},
    {"event_id": 4624, "user": "svc_backup", "src_host": "WS002", "dst_host": "FS01", "logon_type": 3, "time": "2024-01-15 10:03:00"},  # Service account from workstation
]

df = pd.DataFrame(AUTH_EVENTS)
df['time'] = pd.to_datetime(df['time'])
print("Authentication Events:")
df

## 2. Detecting Anomalous Authentication

In [ ]:
def detect_auth_anomalies(events: List[Dict]) -> List[Dict]:
    """Detect suspicious authentication patterns."""
    anomalies = []
    user_history = defaultdict(set)  # Track normal source hosts per user
    
    # Known service accounts that shouldn't auth from workstations
    service_accounts = {'svc_backup', 'svc_sql', 'svc_web'}
    
    for event in events:
        user = event['user']
        src = event['src_host']
        dst = event['dst_host']
        
        # Check 1: Service account from workstation
        if user in service_accounts and src.startswith('WS'):
            anomalies.append({
                "type": "service_account_misuse",
                "description": f"Service account {user} authenticated from workstation {src}",
                "severity": "HIGH",
                "technique": "T1078.002"
            })
        
        # Check 2: Server-to-server lateral movement
        if not src.startswith('WS') and not src.startswith('DC'):
            anomalies.append({
                "type": "server_lateral_movement",
                "description": f"Authentication from server {src} to {dst}",
                "severity": "MEDIUM",
                "technique": "T1021"
            })
    
    return anomalies

anomalies = detect_auth_anomalies(AUTH_EVENTS)
for a in anomalies:
    print(f"[{a['severity']}] {a['type']}: {a['description']} ({a['technique']})")

## 3. Graph-Based Attack Path Analysis

In [ ]:
def build_auth_graph(events: List[Dict]) -> nx.DiGraph:
    """Build authentication graph from events."""
    G = nx.DiGraph()
    
    for event in events:
        if event['event_id'] == 4624:  # Successful login
            src = event['src_host']
            dst = event['dst_host']
            
            # Add edge with user as attribute
            if G.has_edge(src, dst):
                G[src][dst]['users'].add(event['user'])
                G[src][dst]['count'] += 1
            else:
                G.add_edge(src, dst, users={event['user']}, count=1)
    
    return G

# Build graph
G = build_auth_graph(AUTH_EVENTS)

# Visualize
plt.figure(figsize=(10, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=2000, font_size=10, arrows=True)
plt.title("Authentication Flow Graph")
plt.show()

# Find potential attack paths
print("\nPotential attack paths from WS001:")
for target in ['DC01', 'DB01']:
    try:
        paths = list(nx.all_simple_paths(G, 'WS001', target))
        for path in paths:
            print(f"  {' -> '.join(path)}")
    except nx.NetworkXNoPath:
        print(f"  No path to {target}")

## 4. Remote Execution Detection

In [ ]:
# Windows events indicating remote execution
REMOTE_EXEC_EVENTS = [
    {"event_id": 7045, "service_name": "PSEXESVC", "host": "DC01", "user": "admin"},  # PsExec
    {"event_id": 4688, "process": "wmiprvse.exe", "cmdline": "wmic /node:FS01", "host": "WS001"},  # WMI
    {"event_id": 4688, "process": "wsmprovhost.exe", "host": "DB01"},  # WinRM
]

REMOTE_EXEC_INDICATORS = {
    "PSEXESVC": {"technique": "T1569.002", "tool": "PsExec"},
    "wmiprvse.exe": {"technique": "T1047", "tool": "WMI"},
    "wsmprovhost.exe": {"technique": "T1021.006", "tool": "WinRM"},
}

def detect_remote_execution(events: List[Dict]) -> List[Dict]:
    """Detect remote execution techniques."""
    detections = []
    
    for event in events:
        for indicator, info in REMOTE_EXEC_INDICATORS.items():
            if indicator in str(event.values()):
                detections.append({
                    "host": event.get('host'),
                    "tool": info['tool'],
                    "technique": info['technique'],
                    "event": event
                })
    
    return detections

remote_exec = detect_remote_execution(REMOTE_EXEC_EVENTS)
print("Remote Execution Detections:")
for r in remote_exec:
    print(f"  {r['host']}: {r['tool']} ({r['technique']})")

## 5. First-Time Access Detection

In [ ]:
class FirstTimeAccessDetector:
    """Detect first-time user access to hosts."""
    
    def __init__(self):
        self.baseline = defaultdict(set)  # user -> set of hosts
    
    def train(self, events: List[Dict]):
        """Build baseline from historical data."""
        for event in events:
            if event['event_id'] == 4624:
                self.baseline[event['user']].add(event['dst_host'])
    
    def detect(self, event: Dict) -> bool:
        """Check if this is first-time access."""
        if event['event_id'] != 4624:
            return False
        return event['dst_host'] not in self.baseline.get(event['user'], set())

# Demo
detector = FirstTimeAccessDetector()
detector.train(AUTH_EVENTS[:3])  # Train on first 3 events

# Check remaining events
print("First-time access checks:")
for event in AUTH_EVENTS[3:]:
    if event['event_id'] == 4624:
        is_first = detector.detect(event)
        print(f"  {event['user']} -> {event['dst_host']}: {'FIRST TIME!' if is_first else 'Known'}")

## Key Takeaways

1. **Auth Anomalies**: Service accounts, server-to-server movement
2. **Graph Analysis**: Visualize and analyze attack paths
3. **Remote Execution**: PsExec, WMI, WinRM indicators
4. **First-Time Access**: Baseline-based anomaly detection

## Next Steps
- **Lab 16**: Threat Actor Profiling
- **Lab 17**: Adversarial ML